<a href="https://colab.research.google.com/github/loregi01/Cart/blob/main/Cart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import gym
from gym import Env
import numpy as np
import random
from collections import defaultdict

Let's define the two policies we will use, the Random policy for the first attempt and the Learning Policy for the second attempt.

In [18]:
class RandomPolicy:
    def __init__(self, n_actions):
        self.n_actions = n_actions
    def __call__(self, obs) -> int:
        return random.randint(0, self.n_actions - 1)

class GreedyPolicy:
    def __init__(self, Q):
        self.Q = Q
    def __call__(self, obs) -> int:
        return np.argmax(self.Q[obs])

class LearningPolicy:
    def __init__(self, Q):
        self.Q = Q
        self.n_actions = len(Q[0])
    def __call__(self, obs, eps: float) -> int:
        greedy = random.random() > eps
        if greedy:
            return np.argmax(self.Q[obs])
        else:
            return random.randint(0, self.n_actions - 1)


First we try the environment using the random policy, this means that we don't use learning but we perform randomly a new action at each time

We can define a function for the first attempt

In [19]:
def random_attempt (env: gym.Env, policy, discount_factor: float, n_episodes: int, render=False) -> float:

  sum = 0.0
  observation = env.reset()
  discounting = 1

  for episode in range(0, n_episodes):

    discounting = 1

    print ("Episode: " + str(episode))

    observation = env.reset()
    done = False

    while not done :
      if done:
        observation = env.reset()
        discounting = 1
      obs = observation[0] + observation[1]

      selected_action = policy(obs)

      # let's analyze the effects of our actions
      # again we can ignore the info parameter
      observation, reward, done, info = env.step(selected_action)

      sum += reward * discounting
      discounting *= discount_factor

  return sum/n_episodes

In [27]:
learning_rate = 0.01
n_episodes = 20000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)
final_epsilon = 0.1
eps = start_epsilon
discount_factor = 0.95

env = gym.make('MountainCar-v0')

random_trial = random_attempt (env, RandomPolicy(env.action_space.n), discount_factor, n_episodes, render = True)

print(random_trial)

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Output streaming troncato alle ultime 5000 righe.
Episode: 15001
Episode: 15002
Episode: 15003
Episode: 15004
Episode: 15005
Episode: 15006
Episode: 15007
Episode: 15008
Episode: 15009
Episode: 15010
Episode: 15011
Episode: 15012
Episode: 15013
Episode: 15014
Episode: 15015
Episode: 15016
Episode: 15017
Episode: 15018
Episode: 15019
Episode: 15020
Episode: 15021
Episode: 15022
Episode: 15023
Episode: 15024
Episode: 15025
Episode: 15026
Episode: 15027
Episode: 15028
Episode: 15029
Episode: 15030
Episode: 15031
Episode: 15032
Episode: 15033
Episode: 15034
Episode: 15035
Episode: 15036
Episode: 15037
Episode: 15038
Episode: 15039
Episode: 15040
Episode: 15041
Episode: 15042
Episode: 15043
Episode: 15044
Episode: 15045
Episode: 15046
Episode: 15047
Episode: 15048
Episode: 15049
Episode: 15050
Episode: 15051
Episode: 15052
Episode: 15053
Episode: 15054
Episode: 15055
Episode: 15056
Episode: 15057
Episode: 15058
Episode: 15059
Episode: 15060
Episode: 15061
Episode: 15062
Episode: 15063
Episo

In [21]:
def learning_attempt (env: gym.Env, learning_rate: float, discount_factor: float, n_episodes: int):

  Q = defaultdict(lambda: np.zeros(env.action_space.n))
  policy = LearningPolicy(Q)

  for episode in range(0, n_episodes):
    observation, info = env.reset()
    done = False
    flag = True
    flag2 = True
    eps = (n_episodes - episode) / n_episodes

    while not done :
      if done:
          observation = env.reset()

      if flag2:
        selected_action = policy(observation, eps)
        flag2 = False
      else:
        selected_action = policy(observation[0]+observation[1], eps)

      new_observation, reward, done, info = env.step(selected_action)

      if flag:
        obs1 = observation
        obs2 = new_observation[0] + new_observation [1]
        flag = False
      else:
        obs1 = observation[0] + observation[1]
        obs2 = new_observation[0] + new_observation [1]

      Q[obs1][selected_action] += learning_rate * (reward + discount_factor * np.max(Q[obs2]) - Q[obs1][selected_action])

      observation = new_observation
    eps = max(final_epsilon, eps - epsilon_decay)

  return Q

In [28]:
q_table = learning_attempt (env, learning_rate, discount_factor, n_episodes)

In [ ]:
training = random_attempt(env, GreedyPolicy(q_table), learning_rate, n_episodes, render = True)
print(training)

Output streaming troncato alle ultime 5000 righe.
Episode: 6461
Episode: 6462
Episode: 6463
Episode: 6464
Episode: 6465
Episode: 6466
Episode: 6467
Episode: 6468
Episode: 6469
Episode: 6470
Episode: 6471
Episode: 6472
Episode: 6473
Episode: 6474
Episode: 6475
Episode: 6476
Episode: 6477
Episode: 6478
Episode: 6479
Episode: 6480
Episode: 6481
Episode: 6482
Episode: 6483
Episode: 6484
Episode: 6485
Episode: 6486
Episode: 6487
Episode: 6488
Episode: 6489
Episode: 6490
Episode: 6491
Episode: 6492
Episode: 6493
Episode: 6494
Episode: 6495
Episode: 6496
Episode: 6497
Episode: 6498
Episode: 6499
Episode: 6500
Episode: 6501
Episode: 6502
Episode: 6503
Episode: 6504
Episode: 6505
Episode: 6506
Episode: 6507
Episode: 6508
Episode: 6509
Episode: 6510
Episode: 6511
Episode: 6512
Episode: 6513
Episode: 6514
Episode: 6515
Episode: 6516
Episode: 6517
Episode: 6518
Episode: 6519
Episode: 6520
Episode: 6521
Episode: 6522
Episode: 6523
Episode: 6524
Episode: 6525
Episode: 6526
Episode: 6527
Episode: 652